In [ ]:
import numpy as np
import pandas as pd
from numba import njit

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


### Разминка

1. Сгенерируйте массив A из N=1млн случайных целых чисел на отрезке от 0 до 1000. Пусть B[i] = A[i] + 100. Посчитайте среднее значение массива B.


In [ ]:
def b_mean():
    A = np.random.randint(1000,size = 1000000)
    B = A +100
    return B.mean()
b_mean()
%timeit b_mean()

5.77 ms ± 40.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец key, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце key указаны первые 5 английских букв.


In [ ]:
def rand_numb_array(): 
    return np.random.randint(1000,size = 2000000)
abc = [chr(np.random.randint(low = 63,high = 91)) for i in range(2000000)]
data = {'key':abc,'1': rand_numb_array(),'2':rand_numb_array(), '3':rand_numb_array()}
df = pd.DataFrame(data)
df.loc[df['key'].isin(["A","B","C","D","E"])]

,key,1,2,3
5,E,563,972,578
16,D,659,86,678
25,C,488,520,254
28,A,827,49,903
41,B,707,879,288
...,...,...,...,...
1999974,D,260,944,435
1999977,B,49,195,792
1999988,A,385,785,464
1999992,D,420,35,501


1. В файлах recipes_short.csv и reviews_short.csv находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде pd.DataFrame с названиями recipes и reviews. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.


In [ ]:
recipes = pd.read_csv("recipes_sample.csv")
reviews = pd.read_csv("reviews_sample.csv")
reviews = reviews.rename(columns={'Unnamed: 0': 'Index'})
reviews['date'] = pd.to_datetime(reviews['date'])
recipes['submitted'] = pd.to_datetime(recipes['submitted'])

reviews

,Index,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


2. Реализуйте несколько вариантов функции подсчета среднего значения столбца rating из таблицы reviews для отзывов, оставленных в 2010 году.    
С использованием метода DataFrame.iterrows и без использования срезов таблицы    
С использованием метода DataFrame.iterrows и с использованием срезов таблицы    
С использованием метода DataFrame.mean    
Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [ ]:
def mean_rating_1(dataset: pd.DataFrame):
    return dataset[dataset['date'].dt.year == 2010]['rating'].mean()

mean_rating_1(reviews)
%timeit mean_rating_1(reviews)

9.46 ms ± 69.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
def mean_rating_2(dataset: pd.DataFrame):
    count = 0
    total = 0
    for i,r in dataset.iterrows():
        if r['date'].year == 2010:
            count += 1
            total += r['rating']
    return total/count
mean_rating_2(reviews)

4.703745743473326

In [ ]:
%timeit mean_rating_2(reviews)


3.86 s ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


3. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер line_profiler. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.
(*). Сможете ли вы ускорить работу функции 2В?

In [ ]:
%%writefile prof_test.py


def mean_rating2(dataset):
    count = 0
    total = 0
    for i, r in dataset.iterrows():
        if r['date'].year == 2010:
            count += 1
            total += r['rating']
    return total/count


Writing prof_test.py


In [ ]:
from prof_test import mean_rating2

In [ ]:
%lprun -f mean_rating2 mean_rating2(reviews)

Timer unit: 1e-07 s

Total time: 23.6437 s
File: c:\Users\nikit\Documents\University\TOD\Lab8\prof_test.py
Function: mean_rating2 at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def mean_rating2(dataset):
     4         1         12.0     12.0      0.0      count = 0
     5         1          7.0      7.0      0.0      total = 0
     6    126696  206589894.0   1630.6     87.4      for i, r in dataset.iterrows():
     7    114602   27451281.0    239.5     11.6          if r['date'].year == 2010:
     8     12094     100039.0      8.3      0.0              count += 1
     9     12094    2295606.0    189.8      1.0              total += r['rating']
    10         1         12.0     12.0      0.0      return total/count

Timer unit: 1e-07 s

Total time: 23.6437 s
File: c:\Users\nikit\Documents\University\TOD\Lab8\prof_test.py
Function: mean_rating2 at line 3

Line       Hits         Time  Per Hit   % Time  Line Contents

     3                                           def mean_rating2(dataset):
     4         1         12.0     12.0      0.0      count = 0
     5         1          7.0      7.0      0.0      total = 0
     6    126696  206589894.0   1630.6     87.4      for i, r in dataset.iterrows():
     7    114602   27451281.0    239.5     11.6          if r['date'].year == 2010:
     8     12094     100039.0      8.3      0.0              count += 1
     9     12094    2295606.0    189.8      1.0              total += r['rating']
    10         1         12.0     12.0      0.0      return total/count

4. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [ ]:
def find_word1(word,dataset: pd.DataFrame):
    count = 0
    for i,r in dataset.iterrows():
        review = str(r['review'])
        if word in review.split():
            count += 1
    return count


%timeit find_word1('Last', reviews)

5.18 s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def find_word2(word,dataset: pd.DataFrame):
    count = 0
    for row in dataset.itertuples(index=False):
        review = str(row.review).split()
        if word in review:
            count += 1
    return count
%timeit find_word2('Last', reviews)

881 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


5. Напишите несколько версий функции MAPE (см. MAPE) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва.    
Без использования массивов numpy и numba    
Без использования массивов numpy, но с использованием numba    
С использованием массивов numpy, но без использования numba    
C использованием массивов numpy и numba    
Измерьте время выполнения каждой из реализаций.    

Замечание: удалите из выборки отзывы с нулевым рейтингом.

In [ ]:
#1 Без использования массивов numpy и numba
reviews.drop(index=list(reviews[reviews['rating'] == 0].index),inplace=True)
def mape1(dataset: pd.DataFrame):
    recipes = {}
    for row in dataset.itertuples(index=False):
        if row.recipe_id in recipes.keys():
            recipes[row.recipe_id][0] += row.rating
            recipes[row.recipe_id][1] += 1
        else:
            recipes[row.recipe_id] = [row.rating,1]
    avg_rat = {}
    for key,value in recipes.items():
        avg_rat[key] = value[0]/value[1]

    avg_dev = {key:[] for key in recipes.keys()}
    for row in dataset.itertuples(index=False):
        avg_dev[row.recipe_id].append(abs((row.rating - avg_rat[row.recipe_id])/row.rating))

    arr = {}
    for key,value in avg_dev.items():
        arr[key] = sum(value)/len(value) * 100
    return arr

%timeit mape1(reviews)

379 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#2 Без использования массивов numpy, но с использованием numba

def mape2(dataset: pd.DataFrame):
    recipes = {}
    for row in dataset.itertuples(index=False):
        if row.recipe_id in recipes.keys():
            recipes[row.recipe_id][0] += row.rating
            recipes[row.recipe_id][1] += 1
        else:
            recipes[row.recipe_id] = [row.rating,1]
    avg_rat = {}
    for key,value in recipes.items():
        avg_rat[key] = value[0]/value[1]

    avg_dev = {key:[] for key in recipes.keys()}
    for row in dataset.itertuples(index=False):
        avg_dev[row.recipe_id].append(abs((row.rating - avg_rat[row.recipe_id])/row.rating))

    arr = {}
    for key,value in avg_dev.items():
        arr[key] = sum(value)/len(value) * 100
    return arr

%timeit mape2(reviews)

384 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# 3 c использованием массивов numpy, но без использования numba 
def mape3(dataset: pd.DataFrame):
    recipes = {}
    for row in dataset.itertuples(index=False):
        if row.recipe_id in recipes.keys():
            recipes[row.recipe_id][0] += row.rating
            recipes[row.recipe_id][1] += 1
        else:
            recipes[row.recipe_id] = [row.rating,1]
    avg_rat = {}
    for key,value in recipes.items():
        avg_rat[key] = value[0]/value[1]

    arr = {}
    for row in dataset.itertuples(index=False):
        if row.recipe_id not in arr.keys():
            actual = np.array(dataset[dataset['recipe_id'] == row.recipe_id]['rating'])
            pred = np.array([avg_rat[row.recipe_id]] * len(actual))
            arr[row.recipe_id] = np.mean(np.abs((actual - pred) / actual)) * 100
        else:
            continue
    return arr

%timeit mape3(reviews)

10.9 s ± 58.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# 4 c использованием массивов numpy и numba
def mape4(dataset: pd.DataFrame):
    recipes = {}
    for row in dataset.itertuples(index=False):
        if row.recipe_id in recipes.keys():
            recipes[row.recipe_id][0] += row.rating
            recipes[row.recipe_id][1] += 1
        else:
            recipes[row.recipe_id] = [row.rating,1]
    avg_rat = {}
    for key,value in recipes.items():
        avg_rat[key] = value[0]/value[1]

    arr = {}
    for row in dataset.itertuples(index=False):
        if row.recipe_id not in arr.keys():
            actual = np.array(dataset[dataset['recipe_id'] == row.recipe_id]['rating'])
            pred = np.array([avg_rat[row.recipe_id]] * len(actual))
            arr[row.recipe_id] = np.mean(np.abs((actual - pred) / actual)) * 100
        else:
            continue
    return arr

%timeit mape4(reviews)

11.2 s ± 166 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
